In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
import datetime
import boto3
import os
import tempfile



In [2]:
AWS_S3_BUCKET = os.getenv("S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("S3_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("S3_SECRET")

s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)


Credit to Stockanalysis.com for providing this public data to us

In [3]:
page  = requests.get("https://stockanalysis.com/list/biggest-companies/")
page.content


b'<!DOCTYPE html>\n<html lang="en">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1" />\n\t\t\n\t\t<link href="../../_app/immutable/assets/0.1afe3c5b.css" rel="stylesheet">\n\t\t<link href="../../_app/immutable/assets/___StockTable.f29087bf.css" rel="stylesheet"><title>All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis</title><!-- HEAD_svelte-1ipx2c_START --><meta name="description" content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily."><meta name="robots" content="index, follow"><link rel="canonical" href="https://stockanalysis.com/list/biggest-companies/"><meta property="og:title" content="All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis"><meta property="og:description" content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily."><meta prop

Scrape the HTML we need from the page and use the prettify method to organize it

In [4]:
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="../../_app/immutable/assets/0.1afe3c5b.css" rel="stylesheet"/>
  <link href="../../_app/immutable/assets/___StockTable.f29087bf.css" rel="stylesheet"/>
  <title>
   All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis
  </title>
  <!-- HEAD_svelte-1ipx2c_START -->
  <meta content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily." name="description"/>
  <meta content="index, follow" name="robots"/>
  <link href="https://stockanalysis.com/list/biggest-companies/" rel="canonical"/>
  <meta content="All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis" property="og:title"/>
  <meta content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily." property="og:description"/>

Now it's just a matter of turning the data into the columns we need

In [5]:
stocks = soup.find(class_='svelte-1tv1ofl')
stocks

<div class="styled-scroll overflow-x-auto svelte-1tv1ofl"><table class="symbol-table svelte-1tv1ofl" id="main-table"><thead class="svelte-1tv1ofl"><tr class="svelte-1tv1ofl"><th class="svelte-1tv1ofl" draggable="true" id="no">No.
					</th><th class="sym svelte-1tv1ofl" draggable="true" id="s">Symbol
					</th><th class="slw svelte-1tv1ofl" draggable="true" id="n">Company Name
					</th><th class="svelte-1tv1ofl" draggable="true" id="marketCap">
<span class="svelte-1tv1ofl">Market Cap <!-- HTML_TAG_START --><svg class="mt-0.5 w-4 h-4" fill="currentColor" style="max-width:40px" viewbox="0 0 20 20"><path clip-rule="evenodd" d="M5.293 7.293a1 1 0 011.414 0L10 10.586l3.293-3.293a1 1 0 111.414 1.414l-4 4a1 1 0 01-1.414 0l-4-4a1 1 0 010-1.414z" fill-rule="evenodd"></path></svg><!-- HTML_TAG_END --></span>
</th><th class="svelte-1tv1ofl" draggable="true" id="price">Stock Price
					</th><th class="svelte-1tv1ofl" draggable="true" id="change">% Change
					</th><th class="tr svelte-1tv1ofl" dr

In [6]:
names = stocks.find_all(class_='slw svelte-1tv1ofl')
names

[<th class="slw svelte-1tv1ofl" draggable="true" id="n">Company Name
 					</th>,
 <td class="slw svelte-1tv1ofl">Apple Inc.</td>,
 <td class="slw svelte-1tv1ofl">Microsoft Corporation</td>,
 <td class="slw svelte-1tv1ofl">Alphabet Inc.</td>,
 <td class="slw svelte-1tv1ofl">Amazon.com, Inc.</td>,
 <td class="slw svelte-1tv1ofl">NVIDIA Corporation</td>,
 <td class="slw svelte-1tv1ofl">Tesla, Inc.</td>,
 <td class="slw svelte-1tv1ofl">Berkshire Hathaway Inc.</td>,
 <td class="slw svelte-1tv1ofl">Meta Platforms, Inc.</td>,
 <td class="slw svelte-1tv1ofl">Taiwan Semiconductor Manufacturing Company Limited</td>,
 <td class="slw svelte-1tv1ofl">Visa Inc.</td>,
 <td class="slw svelte-1tv1ofl">UnitedHealth Group Incorporated</td>,
 <td class="slw svelte-1tv1ofl">Exxon Mobil Corporation</td>,
 <td class="slw svelte-1tv1ofl">Eli Lilly and Company</td>,
 <td class="slw svelte-1tv1ofl">Johnson &amp; Johnson</td>,
 <td class="slw svelte-1tv1ofl">JPMorgan Chase &amp; Co.</td>,
 <td class="slw svelte

In [7]:
symbols = stocks.find_all(class_='sym svelte-1tv1ofl')
symbols

[<th class="sym svelte-1tv1ofl" draggable="true" id="s">Symbol
 					</th>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/aapl/">AAPL</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/msft/">MSFT</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/googl/">GOOGL</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/amzn/">AMZN</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/nvda/">NVDA</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/tsla/">TSLA</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/brk.b/">BRK.B</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/meta/">META</a><!-- HTML_TAG_END --></td>,


In [8]:
prices = stocks.find_all(class_="svelte-1tv1ofl")
prices

[<table class="symbol-table svelte-1tv1ofl" id="main-table"><thead class="svelte-1tv1ofl"><tr class="svelte-1tv1ofl"><th class="svelte-1tv1ofl" draggable="true" id="no">No.
 					</th><th class="sym svelte-1tv1ofl" draggable="true" id="s">Symbol
 					</th><th class="slw svelte-1tv1ofl" draggable="true" id="n">Company Name
 					</th><th class="svelte-1tv1ofl" draggable="true" id="marketCap">
 <span class="svelte-1tv1ofl">Market Cap <!-- HTML_TAG_START --><svg class="mt-0.5 w-4 h-4" fill="currentColor" style="max-width:40px" viewbox="0 0 20 20"><path clip-rule="evenodd" d="M5.293 7.293a1 1 0 011.414 0L10 10.586l3.293-3.293a1 1 0 111.414 1.414l-4 4a1 1 0 01-1.414 0l-4-4a1 1 0 010-1.414z" fill-rule="evenodd"></path></svg><!-- HTML_TAG_END --></span>
 </th><th class="svelte-1tv1ofl" draggable="true" id="price">Stock Price
 					</th><th class="svelte-1tv1ofl" draggable="true" id="change">% Change
 					</th><th class="tr svelte-1tv1ofl" draggable="true" id="revenue">Revenue
 					</th></t

In [9]:
stock_names = []
for stock in names:
    stock_names.append(stock.get_text())

stock_names

['Company Name\n\t\t\t\t\t',
 'Apple Inc.',
 'Microsoft Corporation',
 'Alphabet Inc.',
 'Amazon.com, Inc.',
 'NVIDIA Corporation',
 'Tesla, Inc.',
 'Berkshire Hathaway Inc.',
 'Meta Platforms, Inc.',
 'Taiwan Semiconductor Manufacturing Company Limited',
 'Visa Inc.',
 'UnitedHealth Group Incorporated',
 'Exxon Mobil Corporation',
 'Eli Lilly and Company',
 'Johnson & Johnson',
 'JPMorgan Chase & Co.',
 'Walmart Inc.',
 'Novo Nordisk A/S',
 'Mastercard Incorporated',
 'The Procter & Gamble Company',
 'Broadcom Inc.',
 'The Home Depot, Inc.',
 'Chevron Corporation',
 'Oracle Corporation',
 'ASML Holding N.V.',
 'Merck & Co., Inc.',
 'The Coca-Cola Company',
 'PepsiCo, Inc.',
 'AbbVie Inc.',
 'Bank of America Corporation',
 'Costco Wholesale Corporation',
 'AstraZeneca PLC',
 'Alibaba Group Holding Limited',
 'Pfizer Inc.',
 'Novartis AG',
 'BHP Group Limited',
 "McDonald's Corporation",
 'Salesforce, Inc.',
 'Cisco Systems, Inc.',
 'Adobe Inc.',
 'Shell plc',
 'Toyota Motor Corporation

In [10]:
stock_symbols = []
for stock in symbols:
    stock_symbols.append(stock.get_text())

stock_symbols

['Symbol\n\t\t\t\t\t',
 'AAPL',
 'MSFT',
 'GOOGL',
 'AMZN',
 'NVDA',
 'TSLA',
 'BRK.B',
 'META',
 'TSM',
 'V',
 'UNH',
 'XOM',
 'LLY',
 'JNJ',
 'JPM',
 'WMT',
 'NVO',
 'MA',
 'PG',
 'AVGO',
 'HD',
 'CVX',
 'ORCL',
 'ASML',
 'MRK',
 'KO',
 'PEP',
 'ABBV',
 'BAC',
 'COST',
 'AZN',
 'BABA',
 'PFE',
 'NVS',
 'BHP',
 'MCD',
 'CRM',
 'CSCO',
 'ADBE',
 'SHEL',
 'TM',
 'TMO',
 'AMD',
 'ACN',
 'FMX',
 'NFLX',
 'LIN',
 'ABT',
 'DHR',
 'DIS',
 'CMCSA',
 'NKE',
 'WFC',
 'TMUS',
 'TXN',
 'SAP',
 'HSBC',
 'NEE',
 'VZ',
 'UPS',
 'TTE',
 'RTX',
 'MS',
 'PM',
 'BMY',
 'INTC',
 'HON',
 'BA',
 'QCOM',
 'SNY',
 'RY',
 'AXP',
 'COP',
 'UL',
 'LOW',
 'SPGI',
 'UNP',
 'IBM',
 'CAT',
 'HDB',
 'SONY',
 'INTU',
 'AMGN',
 'LMT',
 'GE',
 'PLD',
 'T',
 'AMAT',
 'SBUX',
 'DE',
 'GS',
 'MDT',
 'ELV',
 'BUD',
 'NOW',
 'ISRG',
 'TD',
 'BX',
 'SYK',
 'RIO',
 'BP',
 'BLK',
 'PDD',
 'MDLZ',
 'GILD',
 'BKNG',
 'SCHW',
 'DEO',
 'C',
 'CVS',
 'ADI',
 'TJX',
 'ADP',
 'AMT',
 'EQNR',
 'MMC',
 'PBR',
 'VRTX',
 'MUFG',
 'LRCX',

In [11]:
count = 0
count2 = 0
stock_prices = []
for stock in prices:
    if count >= 13:
        stock_prices.append(stock.get_text())
    count += 1

stock_prices2 = []
for stock in stock_prices:
    if count2 == 4:
        stock_prices2.append(stock)
    count2 += 1
    if count2 > 7:
        count2 = 0

stock_prices2

['180.56',
 '325.19',
 '122.15',
 '124.22',
 '384.99',
 '234.85',
 '335.97',
 '264.58',
 '99.93',
 '223.02',
 '490.27',
 '108.17',
 '445.66',
 '160.41',
 '140.72',
 '152.15',
 '159.46',
 '367.63',
 '146.43',
 '804.09',
 '301.10',
 '159.22',
 '107.46',
 '719.95',
 '110.32',
 '60.38',
 '182.10',
 '137.67',
 '29.23',
 '520.08',
 '74.26',
 '86.15',
 '39.09',
 '100.68',
 '60.02',
 '285.82',
 '209.52',
 '49.66',
 '438.96',
 '58.84',
 '146.85',
 '515.28',
 '121.04',
 '307.25',
 '104.84',
 '409.25',
 '362.18',
 '100.82',
 '235.46',
 '92.53',
 '40.46',
 '106.21',
 '42.37',
 '130.30',
 '172.26',
 '133.27',
 '38.48',
 '74.90',
 '35.43',
 '171.50',
 '59.49',
 '99.08',
 '86.30',
 '92.19',
 '65.31',
 '31.81',
 '198.13',
 '218.10',
 '116.41',
 '51.55',
 '92.37',
 '171.15',
 '104.55',
 '50.17',
 '210.49',
 '382.17',
 '200.95',
 '134.46',
 '234.44',
 '64.67',
 '97.41',
 '423.32',
 '220.77',
 '464.23',
 '106.10',
 '124.93',
 '16.07',
 '136.19',
 '99.12',
 '380.85',
 '335.47',
 '83.10',
 '466.36',
 '55.0

In [12]:
count3 = 0
market_cap = []
for stock in stock_prices:
    if count3 == 3:
        market_cap.append(stock)
    count3 += 1
    if count3 > 7:
        count3 = 0

market_cap

['2,840.13B',
 '2,418.47B',
 '1,554.04B',
 '1,274.85B',
 '951.20B',
 '744.39B',
 '735.76B',
 '678.05B',
 '518.28B',
 '467.16B',
 '456.44B',
 '437.41B',
 '423.03B',
 '416.65B',
 '411.22B',
 '409.80B',
 '357.70B',
 '345.57B',
 '345.15B',
 '335.29B',
 '302.72B',
 '301.70B',
 '290.09B',
 '284.12B',
 '279.96B',
 '261.03B',
 '250.88B',
 '242.78B',
 '232.94B',
 '230.48B',
 '230.16B',
 '220.73B',
 '220.70B',
 '213.38B',
 '211.16B',
 '208.62B',
 '204.08B',
 '202.45B',
 '201.38B',
 '200.75B',
 '199.19B',
 '198.75B',
 '194.93B',
 '194.06B',
 '187.57B',
 '181.98B',
 '177.30B',
 '175.30B',
 '173.70B',
 '169.06B',
 '168.33B',
 '163.24B',
 '158.94B',
 '156.37B',
 '156.33B',
 '155.47B',
 '153.05B',
 '151.55B',
 '148.91B',
 '147.28B',
 '144.82B',
 '144.78B',
 '144.15B',
 '143.10B',
 '137.21B',
 '132.72B',
 '131.90B',
 '131.21B',
 '129.61B',
 '128.73B',
 '128.33B',
 '127.17B',
 '126.51B',
 '126.27B',
 '123.34B',
 '122.60B',
 '122.52B',
 '122.04B',
 '121.06B',
 '120.49B',
 '120.25B',
 '118.57B',
 '117.96

In [13]:
count4 = 0
percent_change = []
for stock in stock_prices:
    if count4 == 5:
        percent_change.append(stock)
    count4 += 1
    if count4 > 7:
        count4 = 0

percent_change

['1.54%',
 '0.56%',
 '-0.29%',
 '2.44%',
 '2.73%',
 '4.58%',
 '0.16%',
 '0.37%',
 '-0.36%',
 '-1.00%',
 '1.69%',
 '-0.33%',
 '2.13%',
 '1.19%',
 '0.02%',
 '1.43%',
 '1.23%',
 '-1.01%',
 '1.12%',
 '1.45%',
 '-0.20%',
 '-0.38%',
 '2.10%',
 '1.37%',
 '1.57%',
 '0.27%',
 '1.10%',
 '0.86%',
 '-0.86%',
 '1.73%',
 '1.30%',
 '1.01%',
 '0.51%',
 '2.21%',
 '2.20%',
 '1.39%',
 '1.84%',
 '-0.25%',
 '4.93%',
 '1.09%',
 '1.06%',
 '-1.06%',
 '2.72%',
 '-1.22%',
 '0.83%',
 '2.37%',
 '-0.03%',
 '-0.83%',
 '-0.87%',
 '0.01%',
 '0.27%',
 '-0.83%',
 '0.70%',
 '2.96%',
 '0.15%',
 '1.19%',
 '0.25%',
 '0.98%',
 '0.47%',
 '-',
 '0.93%',
 '0.50%',
 '-0.23%',
 '-0.07%',
 '0.23%',
 '1.69%',
 '-0.13%',
 '2.91%',
 '1.06%',
 '1.32%',
 '0.38%',
 '-0.23%',
 '-0.25%',
 '0.48%',
 '-0.34%',
 '2.13%',
 '-0.25%',
 '0.06%',
 '-0.47%',
 '-0.35%',
 '-0.28%',
 '0.23%',
 '-0.22%',
 '0.54%',
 '0.02%',
 '-2.44%',
 '-0.19%',
 '0.87%',
 '1.23%',
 '-0.25%',
 '-0.08%',
 '0.25%',
 '1.35%',
 '-0.16%',
 '1.31%',
 '1.16%',
 '1.60%',
 '1

In [14]:
count5 = 0
revenue = []
for stock in stock_prices:
    if count5 == 6:
        revenue.append(stock)
    count5 += 1
    if count5 > 7:
        count5 = 0

revenue

['385.10B',
 '207.59B',
 '284.61B',
 '524.90B',
 '25.88B',
 '86.04B',
 '353.41B',
 '117.35B',
 '74.80B',
 '30.98B',
 '335.94B',
 '409.74B',
 '27.69B',
 '96.26B',
 '136.33B',
 '622.02B',
 '27.57B',
 '22.82B',
 '80.97B',
 '34.41B',
 '155.75B',
 '242.67B',
 '47.96B',
 '26.51B',
 '57.87B',
 '43.49B',
 '88.04B',
 '56.74B',
 '97.98B',
 '235.44B',
 '43.84B',
 '126.45B',
 '92.95B',
 '52.22B',
 '60.28B',
 '23.41B',
 '32.19B',
 '54.90B',
 '18.00B',
 '386.20B',
 '280.62B',
 '43.81B',
 '23.07B',
 '63.14B',
 '34.56B',
 '31.91B',
 '33.35B',
 '41.51B',
 '30.95B',
 '86.98B',
 '120.11B',
 '50.63B',
 '76.92B',
 '79.08B',
 '19.50B',
 '33.51B',
 '61.60B',
 '24.78B',
 '136.19B',
 '98.89B',
 '385.49B',
 '68.57B',
 '53.38B',
 '80.65B',
 '45.85B',
 '56.42B',
 '35.95B',
 '70.54B',
 '41.07B',
 '48.29B',
 '39.79B',
 '52.14B',
 '78.38B',
 '63.91B',
 '95.75B',
 '11.95B',
 '25.07B',
 '60.58B',
 '61.70B',
 '11.71B',
 '87.16B',
 '14.07B',
 '26.19B',
 '66.15B',
 '74.00B',
 '6.52B',
 '121.17B',
 '26.64B',
 '34.00B',
 '

In [15]:
data = {
    'Name': stock_names[1 : 301],
    'Symbol': stock_symbols[1 : 301],
    'Price': stock_prices2[0 : 300],
    'Market_Cap': market_cap[0 : 300],
    'Percent_Change': percent_change[0 : 300],
    'Revenue': revenue[0 : 300],
}

df = pd.DataFrame(data)
df['Timestamp'] = datetime.datetime.utcnow()
df = df.reset_index()
df = df.rename(columns={'index' : 'id'})
df


,id,Name,Symbol,Price,Market_Cap,Percent_Change,Revenue,Timestamp
0,0,Apple Inc.,AAPL,180.56,"2,840.13B",1.54%,385.10B,2023-06-08 20:09:41.566298
1,1,Microsoft Corporation,MSFT,325.19,"2,418.47B",0.56%,207.59B,2023-06-08 20:09:41.566298
2,2,Alphabet Inc.,GOOGL,122.15,"1,554.04B",-0.29%,284.61B,2023-06-08 20:09:41.566298
3,3,"Amazon.com, Inc.",AMZN,124.22,"1,274.85B",2.44%,524.90B,2023-06-08 20:09:41.566298
4,4,NVIDIA Corporation,NVDA,384.99,951.20B,2.73%,25.88B,2023-06-08 20:09:41.566298
...,...,...,...,...,...,...,...,...
295,295,Haleon plc,HLN,8.26,38.12B,1.54%,13.08B,2023-06-08 20:09:41.566298
296,296,Carrier Global Corporation,CARR,45.61,38.07B,0.01%,21.04B,2023-06-08 20:09:41.566298
297,297,Hilton Worldwide Holdings Inc.,HLT,142.93,37.81B,0.21%,9.35B,2023-06-08 20:09:41.566298
298,298,"IDEXX Laboratories, Inc.",IDXX,452.78,37.56B,0.46%,3.43B,2023-06-08 20:09:41.566298


In [16]:
df.head(5)
df.dtypes
df['Name'] = df['Name'].astype('string')
df['Symbol'] = df['Symbol'].astype('string')
df.replace(',','', regex=True, inplace=True)
df['Price'] = df['Price'].astype(float)
perc_change = df['Percent_Change'].replace('%','', regex=True, inplace=True)
df

,id,Name,Symbol,Price,Market_Cap,Percent_Change,Revenue,Timestamp
0,0,Apple Inc.,AAPL,180.56,2840.13B,1.54,385.10B,2023-06-08 20:09:41.566298
1,1,Microsoft Corporation,MSFT,325.19,2418.47B,0.56,207.59B,2023-06-08 20:09:41.566298
2,2,Alphabet Inc.,GOOGL,122.15,1554.04B,-0.29,284.61B,2023-06-08 20:09:41.566298
3,3,Amazon.com Inc.,AMZN,124.22,1274.85B,2.44,524.90B,2023-06-08 20:09:41.566298
4,4,NVIDIA Corporation,NVDA,384.99,951.20B,2.73,25.88B,2023-06-08 20:09:41.566298
...,...,...,...,...,...,...,...,...
295,295,Haleon plc,HLN,8.26,38.12B,1.54,13.08B,2023-06-08 20:09:41.566298
296,296,Carrier Global Corporation,CARR,45.61,38.07B,0.01,21.04B,2023-06-08 20:09:41.566298
297,297,Hilton Worldwide Holdings Inc.,HLT,142.93,37.81B,0.21,9.35B,2023-06-08 20:09:41.566298
298,298,IDEXX Laboratories Inc.,IDXX,452.78,37.56B,0.46,3.43B,2023-06-08 20:09:41.566298


In [17]:
db = sqlite3.connect('../instance/dev.db')

df.to_sql('stocks', con=db, if_exists='replace', schema='stocks', index=False)


300

In [18]:
tf = tempfile.NamedTemporaryFile()

s3_client.put_object(
    ACL='public-read',
    Bucket=AWS_S3_BUCKET,
    Body=df.to_csv(),
    Key=tf.name
)

{'ResponseMetadata': {'RequestId': '1250Z8JKBYNMWKQ5',
  'HostId': 'NPXnFK6Vabp/Ck5As4k0CXdwAXJxRunGCjN4YGvkNJxhmTNCCAXFRJTdJh8ttw7JHOYZgzLxLB0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'NPXnFK6Vabp/Ck5As4k0CXdwAXJxRunGCjN4YGvkNJxhmTNCCAXFRJTdJh8ttw7JHOYZgzLxLB0=',
   'x-amz-request-id': '1250Z8JKBYNMWKQ5',
   'date': 'Thu, 08 Jun 2023 20:09:43 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d3582a1fbfc6af31acf1fcd90fa08c0a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d3582a1fbfc6af31acf1fcd90fa08c0a"',
 'ServerSideEncryption': 'AES256'}